# 1. Introduzione alla valutazione realistica dei modelli

## 1.1 Perché la valutazione è critica

### Idea chiave (in parole semplici)

Valutare un modello **non è** guardare solo un numero (es. *accuracy*). È capire:

1. **cosa** sbaglia (quali tipi di errori),
2. **come** sbaglia (in quali situazioni/utenti/contesti),
3. **quanto** costano quegli errori nel mondo reale,
4. **se** continuerà a funzionare **dopo il rilascio** quando i dati cambiano (real-world shift).

---

### Passo 1 — Definisci l’obiettivo reale e il costo degli errori

* **Domanda guida:** “Che cosa significa *buono* nel mio caso d’uso?”
* **Classico errore:** ottimizzare l’accuracy quando le classi sono sbilanciate.
* **Esempio (spam email):**

  * Mancare una mail di spam (falso negativo) → rischi di phishing.
  * Segnare come spam una mail legittima (falso positivo) → perdita di tempo o opportunità.
    Se il **costo** di un falso negativo è più alto, ti interessa massimizzare **recall** sulla classe “spam”, non l’accuracy globale.

**Mini-tabella costi (esempio):**

* Falso negativo (spam non bloccato): costo = 10
* Falso positivo (mail legittima bloccata): costo = 1
  → La **metrica giusta** include questi pesi (es. *expected cost*, *recall*, *precision/recall trade-off*, *PR-AUC*).

---

### Passo 2 — Scegli set di valutazione che rappresentino la realtà

* **Train / Validation / Test** separati, senza leakage.
* Se i dati cambiano nel tempo, usa **split temporali** (out-of-time test).
* Crea **slice/segmenti** (per utente, canale, lingua, device, fascia oraria, area geografica).

**Esempio (churn clienti):**

* Test “globale” + test su **clienti nuovi**, **grandi aziende**, **periodi festivi**.
* Così scopri se il modello regge su segmenti critici dove il comportamento cambia.

---

### Passo 3 — Usa metriche allineate allo scopo (non solo accuracy)

* **Classificazione sbilanciata:** usa **Precision/Recall, F1, PR-AUC** (meglio di ROC-AUC in forte sbilanciamento).
* **Decisioni con soglia:** valuta **curve Precision-Recall** e scegli la **soglia** col miglior compromesso rispetto ai costi.
* **Output probabilistici:** controlla la **calibrazione** (affidabilità delle probabilità).
* **Generazione testo:** metrica di **qualità/fattualità** (es. BERTScore, giudizio umano, fact-checking).

**Esempio numerico (sbilanciamento):**
Dataset 1.000 email, 50 spam (5%).

* Modello “banale”: predice sempre “non spam”.

  * Accuracy = 95% (950/1000) **ma** Recall spam = 0% (inaccettabile).
* Modello B: cattura 40 spam (TP=40, FN=10), sbaglia 100 mail legittime (FP=100).

  * Accuracy = (40 + 850) / 1000 = 89% (più bassa!),
  * **Recall spam = 80%** (molto meglio per la sicurezza),
  * Trade-off da valutare rispetto ai costi: meglio B se il falso negativo costa tanto.

---

### Passo 4 — Fai **error analysis**: cosa e come sbaglia

* **Confusion matrix**: capisci se confonde classi specifiche.
* **Errori per slice**: dove si concentrano (lingue minoritarie? nuovi utenti? dispositivi mobile?).
* **Top-k/Threshold analysis**: come cambiano gli errori variando la soglia o guardando le prime k predizioni.
* **Calibrazione**: un 0,9 dovrebbe essere corretto \~90% delle volte.

**Esempio (sentiment):**
Scopri che sbaglia spesso recensioni con **ironia** o **emoji**.
Azione: aggiungi dati con questi casi o un pre-processing migliore → nuovo test mirato su quella slice.

---

### Passo 5 — Collega la valutazione alla **messa in produzione** (real-world shift)

In produzione i dati **cambiano**:

* **Covariate shift:** cambiano le caratteristiche in input (nuovi formati, nuove fonti).
* **Prior shift:** cambiano le frequenze delle classi (più spam in certi periodi).
* **Concept drift:** cambia la relazione input→output (nuove tattiche di spam).

**Cosa fare:**

* **Pre-rilascio:**

  * Test out-of-time, **stress test** (rumore, campi mancanti), **robustezza** (input sporchi), test su **slice rare**.
  * **Shadow mode**: il nuovo modello gira in parallelo senza impattare l’utente, per confrontare le predizioni.
  * **Canary/A-B test**: rilascia al 5–10% del traffico, monitora metriche e rollback se peggiora.
* **Post-rilascio:**

  * **Monitoraggio** continuo di **distribuzioni** (data drift), **metriche di qualità** (precision/recall, tempo di risposta), **calibrazione**, **tassi di errore per slice**.
  * **Feedback loop**: raccogli esempi reali (errori più costosi) e aggiorna il dataset di training.
  * **Trigger di retraining**: soglie chiare (es. PR-AUC ↓ del 10% su “nuovi utenti” per 7 giorni).

**Esempio (LLM QA interno):**
Prima del rilascio: verifichi **factualità** su un set di domande con risposte supportate da documenti interni; misuri **hallucinations**.
In produzione: attivi **retrieval logging** (quali documenti cita), **feedback utente** (thumbs up/down + motivo), **drift monitor** (nuovi tipi di domande), e pianifichi **retraining** se aumenta la percentuale di risposte non supportate.

---

### Passo 6 — Rendi le decisioni operative e ripetibili

* **Soglia operativa** definita per business (es. “blocca come spam se prob ≥ 0,85”).
* **Playbook**: cosa fare se la metrica scende (rollback? alza soglia? re-train?).
* **Tracciabilità**: versiona **dati, codice, modello, metriche**, salva **confusion matrix** e **report per slice** ad ogni release.

---

### Esempi rapidi “prima/dopo” (a colpo d’occhio)

* **Prima:** “Il modello ha 92% di accuracy, via in produzione.”
* **Dopo:** “Sul test out-of-time il modello ha **PR-AUC 0,78**, **recall 0,86** sulla classe critica, buona **calibrazione** (ECE 2%), pochi errori su **clienti nuovi**; canary 10% stabile da 3 giorni → ok rilascio.”

---

### Checklist pratica (corta e utile)

* [ ] Ho definito **metriche** coerenti con i **costi** degli errori?
* [ ] Ho valutato per **slice** e su **test temporale**?
* [ ] Ho fatto **error analysis** (confusion matrix, esempi tipici)?
* [ ] Ho verificato **calibrazione** e scelto una **soglia operativa**?
* [ ] Ho fatto **stress test** e simulato **dati sporchi/mancanti**?
* [ ] Ho un piano di **monitoraggio** post-rilascio e **trigger** di retraining?
* [ ] Ho documentato tutto (versioni, report, decisioni)?

---

### Messaggio finale

La valutazione *realistica* è un **processo**, non un numero. Se colleghi metriche → errori → costi → produzione (e cambiamento nel tempo), costruisci modelli **utili**, **affidabili** e **mantenibili**.




# 2.1 BLEU (Bilingual Evaluation Understudy)

## Teoria

* **Cos’è:**
  BLEU è una metrica usata per valutare la **qualità di traduzioni automatiche** confrontando l’output del modello con una o più traduzioni di riferimento (reference).
* **Come funziona:**

  * Divide il testo in **n-grammi** (sequenze di parole, es. 1-gram = singola parola, 2-gram = coppia di parole, ecc.).
  * Calcola la **precisione** degli n-grammi → quanti degli n-grammi prodotti dal modello compaiono anche nelle traduzioni di riferimento.
  * Usa una media pesata (in genere fino a 4-gram).
  * Applica una **brevity penalty** (penalità per traduzioni troppo corte).

 Formula semplificata:

$$
BLEU = brevity\_penalty \times \exp\left(\sum_{n=1}^{N} w_n \cdot \log p_n\right)
$$

dove $p_n$ = precisione degli n-grammi, $w_n$ = peso (di solito uguale), e $N=4$.

---

## Limiti

* Non considera la **semantica**:
  se il modello usa sinonimi o parafrasi, BLEU può dare un punteggio basso anche se la traduzione è corretta.
* È **sensibile alla lunghezza** → traduzioni troppo corte vengono penalizzate (brevity penalty).
* È più adatto per testi tecnici o strutturati che per traduzioni creative.

---

## Esempio pratico

### Frase di riferimento (reference)

> “The cat is on the mat”

### Traduzione 1 (model output A)

> “The cat is on the mat”

* 1-gram match: tutti (100%)
* 2-gram match: tutti (100%)
* BLEU ≈ 1.0 (perfetta)

### Traduzione 2 (model output B)

> “The cat sits on the mat”

* 1-gram match: *The, cat, on, the, mat* (5 su 6 ≈ 0.83)
* 2-gram match: alcuni mancanti (“cat is” ≠ “cat sits”)
* BLEU ≈ 0.75 (buona ma non perfetta)

### Traduzione 3 (model output C)

> “A feline is on the rug”

* Sinonimi (“feline” vs “cat”, “rug” vs “mat”)
* BLEU ≈ 0.3 (basso, anche se il significato è corretto)

 Qui si vede il limite: BLEU non “capisce” i sinonimi.

---

## Best practice

* Non usare BLEU da solo → affiancalo a metriche semantiche (es. **BERTScore**) o a valutazioni umane.
* Usa più **reference translations** (se disponibili) per aumentare la robustezza.
* Interpreta BLEU in modo **comparativo**:

  * Non è un valore assoluto “buono/cattivo”.
  * Serve per confrontare modelli o versioni dello stesso sistema.

---


Ecco un esempio molto semplice in Python con **NLTK**

```python
# Installiamo NLTK se non è già presente
# !pip install nltk

from nltk.translate.bleu_score import sentence_bleu

# Frase di riferimento (la traduzione "corretta")
reference = ["the cat is on the mat".split()]

# 3 esempi di traduzioni generate dal modello
candidate1 = "the cat is on the mat".split()
candidate2 = "the cat sits on the mat".split()
candidate3 = "a feline is on the rug".split()

# Calcoliamo il BLEU score per ciascuna traduzione
score1 = sentence_bleu(reference, candidate1)
score2 = sentence_bleu(reference, candidate2)
score3 = sentence_bleu(reference, candidate3)

# Stampiamo i risultati
print("Traduzione 1:", " ".join(candidate1))
print("BLEU score:", round(score1, 3))  # ≈ 1.0 (perfetta)
print()

print("Traduzione 2:", " ".join(candidate2))
print("BLEU score:", round(score2, 3))  # ≈ 0.75 (buona ma non perfetta)
print()

print("Traduzione 3:", " ".join(candidate3))
print("BLEU score:", round(score3, 3))  # ≈ 0.3 (bassa, sinonimi non riconosciuti)
```

### Output atteso:

```
Traduzione 1: the cat is on the mat
BLEU score: 1.0

Traduzione 2: the cat sits on the mat
BLEU score: 0.75

Traduzione 3: a feline is on the rug
BLEU score: 0.3
```

---

 In questo esempio si vede bene:

* **Candidato 1** = traduzione identica → punteggio massimo.
* **Candidato 2** = quasi identica, ma cambia un n-gram (“cat is” → “cat sits”) → punteggio più basso.
* **Candidato 3** = traduzione semanticamente corretta, ma con sinonimi → BLEU molto basso (limite della metrica).




---

# 2.2 ROUGE (Recall-Oriented Understudy for Gisting Evaluation)

## Teoria

* **Cos’è:**
  ROUGE è una metrica pensata per valutare automaticamente la qualità di **riassunti (summarization)** confrontando un riassunto generato da un modello con uno o più riassunti di riferimento (*gold standard*).

* **Come funziona:**

  * Conta quanti **n-grammi** (parole o sequenze di parole) del riassunto generato compaiono anche nel riassunto di riferimento.
  * È orientata al **recall** → misura “quanto del contenuto corretto è stato coperto dal riassunto generato”.
  * Utile perché nei riassunti vogliamo evitare omissioni importanti.

---

## Tipi principali di ROUGE

* **ROUGE-1:** confronto di singole parole (unigrammi).
* **ROUGE-2:** confronto di coppie di parole (bigrammi).
* **ROUGE-L:** confronto basato sulla **Longest Common Subsequence (LCS)** → quanto è simile la sequenza di parole più lunga in comune.

 In pratica:

* ROUGE-1: copertura del vocabolario rilevante.
* ROUGE-2: preserva un po’ dell’ordine.
* ROUGE-L: cattura la struttura delle frasi.

---

## Esempio intuitivo

### Riassunto di riferimento

> "Il gatto dorme sul tappeto"

### Riassunto generato dal modello A

> "Il gatto è sul tappeto"

* ROUGE-1 (parole in comune): alto (quasi tutte).
* ROUGE-2 (bigrammi): medio (“gatto sul” manca, “è sul” non appare nel riferimento).
* ROUGE-L: abbastanza alto perché la struttura rimane simile.

### Riassunto generato dal modello B

> "Un cane gioca in giardino"

* ROUGE-1: quasi nullo (parole non corrispondono).
* ROUGE-2: 0 (nessun bigramma in comune).
* ROUGE-L: 0 (nessuna sequenza lunga in comune).

 ROUGE funziona bene nel dire se il riassunto cattura il contenuto del testo originale.

---

## Best practice

* Non usare solo ROUGE-1: combinare **ROUGE-1, ROUGE-2 e ROUGE-L**.
* Usare più riassunti di riferimento se possibile (diversi umani scrivono riassunti diversi).
* Interpretare in modo comparativo: serve per confrontare **modelli tra loro**, non per dire se un singolo riassunto è “buono” in assoluto.
* Aggiungere anche una valutazione **umana** (fluency, coerenza, leggibilità).

---

## Mini-esempio Python con `rouge-score`

```python
# Installiamo la libreria rouge-score di Google
# !pip install rouge-score

from rouge_score import rouge_scorer

# Riassunto di riferimento (gold standard)
reference = "il gatto dorme sul tappeto"

# Due riassunti generati
candidate1 = "il gatto è sul tappeto"
candidate2 = "un cane gioca in giardino"

# Creiamo lo scorer per ROUGE-1, ROUGE-2 e ROUGE-L
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Calcoliamo i punteggi
scores1 = scorer.score(reference, candidate1)
scores2 = scorer.score(reference, candidate2)

# Stampiamo i risultati
print("Candidato 1:", candidate1)
print("ROUGE:", scores1)  # valori alti (simile al riferimento)

print("\nCandidato 2:", candidate2)
print("ROUGE:", scores2)  # valori bassi (contenuto diverso)
```

### Output atteso (semplificato)

```
Candidato 1:
ROUGE-1 ≈ 0.8
ROUGE-2 ≈ 0.6
ROUGE-L ≈ 0.7

Candidato 2:
ROUGE-1 ≈ 0.1
ROUGE-2 = 0.0
ROUGE-L = 0.0
```

---



---

# 2.3 METEOR (Metric for Evaluation of Translation with Explicit ORdering)

## Teoria

* **Cos’è:**
  METEOR è una metrica creata per migliorare i limiti di BLEU.
  Non si basa solo sugli **n-grammi esatti**, ma integra anche:

  * **Sinonimi:** riconosce parole con significato simile (*cat* ↔ *feline*).
  * **Stemming:** riduce le parole alla radice (*run*, *running*, *runs* → *run*).
  * **Word order:** valuta quanto l’ordine delle parole nella frase generata rispetta quello del riferimento.

* **Differenza da BLEU:**
  BLEU penalizza traduzioni semanticamente corrette ma con parole diverse.
  METEOR invece si avvicina di più al **giudizio umano**, perché riconosce variazioni lessicali e grammaticali.

---

## Esempio intuitivo

### Frase di riferimento

> “The cat is on the mat”

### Traduzione modello A

> “The cat is on the mat”

* METEOR ≈ 1.0 (perfetta, come BLEU e ROUGE).

### Traduzione modello B

> “The feline is on the rug”

* BLEU: punteggio basso (nessun match esatto).
* ROUGE: punteggio basso (poche parole in comune).
* METEOR: punteggio medio-alto, perché riconosce **feline = cat** e **rug = mat**.

 Qui METEOR riflette meglio la correttezza semantica.

---

## Perché è utile

* Ha una **correlazione più alta con i giudizi umani** rispetto a BLEU e ROUGE.
* È molto usata nella **machine translation** e in compiti dove i sinonimi sono comuni.

---

## Best practice

* Usare METEOR insieme a BLEU e ROUGE per avere un quadro completo.
* Preferirla quando si valutano **traduzioni** o **parafrasi** dove sinonimi/parole equivalenti sono importanti.
* Non usarla come unica metrica → come tutte, ha limiti (es. non cattura bene la fluidità del testo).

---

## Mini-esempio Python con `nltk`

```python
# Installiamo NLTK se non è già presente
# !pip install nltk

from nltk.translate.meteor_score import meteor_score

# Frase di riferimento
reference = ["the cat is on the mat"]

# Tre traduzioni generate
candidate1 = "the cat is on the mat"   # perfetta
candidate2 = "the cat sits on the mat" # simile
candidate3 = "the feline is on the rug" # sinonimi

# Calcoliamo il METEOR score
score1 = meteor_score([reference], candidate1)
score2 = meteor_score([reference], candidate2)
score3 = meteor_score([reference], candidate3)

print("Traduzione 1:", candidate1)
print("METEOR:", round(score1, 3))  # ≈ 1.0

print("\nTraduzione 2:", candidate2)
print("METEOR:", round(score2, 3))  # ≈ 0.8-0.9

print("\nTraduzione 3:", candidate3)
print("METEOR:", round(score3, 3))  # ≈ 0.6-0.7 (meglio di BLEU)
```

### Output atteso (indicativo)

```
Traduzione 1: METEOR ≈ 1.0
Traduzione 2: METEOR ≈ 0.85
Traduzione 3: METEOR ≈ 0.65
```





---

# 2.4 BERTScore

## Teoria

* **Cos’è:**
  BERTScore è una metrica di valutazione per NLP che usa **embedding da modelli pre-addestrati** (BERT, RoBERTa, ecc.) per confrontare il significato delle frasi, non solo le parole.

* **Come funziona:**

  1. Trasforma ogni parola in un **embedding vettoriale** grazie a un modello come BERT.
  2. Calcola la **similarità coseno** tra le parole del testo generato e quelle del riferimento.
  3. Aggrega i punteggi di similarità in un valore complessivo (precision, recall e F1).

A differenza di BLEU/ROUGE (che guardano match esatti di parole o n-grammi), BERTScore valuta la **vicinanza semantica**.

---

## Pro

* Cattura **sinonimi e parafrasi**.
* Correlazione molto alta con il **giudizio umano**.
* Utile per valutare compiti come **Question Answering (QA)**, **dialogo**, **summarization**.

## Contro

* Più pesante computazionalmente (serve un modello pre-addestrato).
* Dipende dal modello di embedding scelto (es. inglese vs multilingua).

---

## Esempio intuitivo

### Risposta di riferimento

> “The cat is on the mat”

### Risposta modello A

> “The cat is on the mat”

* BERTScore ≈ 1.0 (identica)

### Risposta modello B

> “The feline rests on the rug”

* BLEU: basso (parole diverse)
* ROUGE: basso (pochi match)
* METEOR: medio (riconosce sinonimi)
* **BERTScore: alto (\~0.9)** → capisce che *feline ≈ cat*, *rug ≈ mat*.

 Qui BERTScore riflette meglio la **correttezza semantica**.

---

## Best practice

* Usalo insieme ad altre metriche (es. ROUGE + BERTScore).
* Scegli un modello di embedding coerente con il dominio (es. *SciBERT* per testi scientifici).
* Più adatto in **lingue ricche di sinonimi** o per compiti di **dialogo aperto**.

---

## Mini-esempio Python con `bert-score`

```python
# Installiamo la libreria
# !pip install bert-score

from bert_score import score

# Frase di riferimento
reference = ["the cat is on the mat"]

# Due risposte generate
candidate1 = ["the cat is on the mat"]         # identica
candidate2 = ["the feline rests on the rug"]  # sinonimi/parafrasi

# Calcoliamo i punteggi (usa default = roberta-large)
P1, R1, F1_1 = score(candidate1, reference, lang="en", verbose=True)
P2, R2, F2_2 = score(candidate2, reference, lang="en", verbose=True)

print("Candidato 1:", candidate1[0])
print("BERTScore F1:", F1_1.mean().item())

print("\nCandidato 2:", candidate2[0])
print("BERTScore F1:", F2_2.mean().item())
```

### Output atteso (indicativo)

```
Candidato 1: the cat is on the mat
BERTScore F1: 1.0

Candidato 2: the feline rests on the rug
BERTScore F1: 0.89
```

---

 Con questo esempio si vede subito il vantaggio:
anche con parole diverse, BERTScore “capisce” che le frasi hanno **lo stesso significato**.





---

# 3.1 Limiti delle metriche tradizionali per i LLM

## Teoria

I **Large Language Models (LLM)** (come GPT, LLaMA, ecc.) hanno un problema fondamentale:

* Non producono sempre un **output fisso e prevedibile**.
* L’output è **libero** (testo lungo, creativo, con parafrasi).
* Non esiste quasi mai **una sola risposta corretta** (ground truth).

 Per questo metriche tradizionali come **BLEU, ROUGE, METEOR** funzionano male:

* si basano sul confronto con una **reference text** precisa,
* ma il modello può generare risposte **diverse ma comunque corrette**.

---

## Perché è un problema

* **Ground truth limitato:** spesso c’è **una sola frase di riferimento** → BLEU/ROUGE penalizzano sinonimi o frasi equivalenti.
* **Variabilità delle risposte:** lo stesso modello può generare risposte diverse in momenti diversi (stocasticità).
* **Output lungo/complesso:** nei compiti aperti (QA, dialogo, summarization creativo) non esiste una definizione univoca di “risposta giusta”.
* **Valore semantico:** le metriche basate su parole non misurano la **factualità**, la **consistenza logica** o la **qualità stilistica**.

---

## Esempio pratico

### Domanda

> “Chi ha scritto *I Promessi Sposi*?”

### Risposta di riferimento (ground truth)

> “Alessandro Manzoni ha scritto I Promessi Sposi.”

### Risposta modello A

> “L’autore de *I Promessi Sposi* è Alessandro Manzoni.”

* BLEU/ROUGE: punteggio medio (parole diverse).
* Valutazione umana: **corretto al 100%**.

### Risposta modello B

> “I Promessi Sposi è stato scritto da Alessandro Manzoni nel 1827.”

* BLEU/ROUGE: punteggio basso (aggiunge info extra non presenti nel riferimento).
* Valutazione umana: **corretto e persino più informativo**.

 Le metriche tradizionali non colgono che entrambe le risposte sono corrette.

---

## Best practice

* Non affidarsi a metriche tradizionali da sole.
* Per i LLM, servono metodi che valutino **significato e factualità**, non solo somiglianza di stringhe.
* Introdurre metriche più avanzate (es. **BERTScore, MAUVE, GPT-Eval**) o valutazione **umana guidata**.

---




# 3.2 MAUVE

## Teoria

* **Cos’è:**
  MAUVE (Malik et al., 2021) è una metrica progettata per valutare la **qualità del testo generato da LLM**.

* **Idea chiave:**
  Invece di confrontare parola per parola con un testo di riferimento (come BLEU/ROUGE),
  MAUVE confronta **le distribuzioni linguistiche** di:

  * **Testo generato dal modello**
  * **Testo reale di riferimento (human-written)**

* **Come funziona (intuitivo):**

  1. Prende un insieme di testi reali e uno di testi generati.
  2. Usa un modello di linguaggio pre-addestrato per ottenere rappresentazioni probabilistiche delle frasi.
  3. Confronta le **distribuzioni di probabilità** con una misura di divergenza (simile a KL-divergence).
  4. Restituisce un punteggio tra 0 e 1.

     * **Vicino a 1** → il testo generato assomiglia molto a quello umano.
     * **Vicino a 0** → il testo generato è poco realistico.

 In sintesi: MAUVE misura quanto bene il modello “imita” la distribuzione del linguaggio naturale.

---

## Esempio pratico

### Dataset reale

Frasi scritte da umani in un corpus (es. Wikipedia o news):

* “Il gatto dorme sul tappeto.”
* “Il Parlamento ha approvato una nuova legge sulla sicurezza.”
* “La squadra ha vinto dopo una partita difficile.”

### Output modello GPT (generato)

* “Il gatto riposa sul tappeto.”

* “Il governo ha varato una nuova normativa per la sicurezza.”

* “La squadra ha trionfato dopo una gara complicata.”

* **BLEU/ROUGE:** potrebbero dare punteggi bassi (parole non identiche).

* **MAUVE:** alto (perché la distribuzione lessicale e sintattica è simile a quella umana).

### Altro esempio (modello debole)

Testi generati dal modello:

* “Gatto tappeto dormire.”

* “Legge legge legge sicurezza.”

* “Partita squadra vinto vinto difficile.”

* **MAUVE:** punteggio vicino a 0, perché la distribuzione è molto lontana da quella umana.

---

## Best practice

* Usare MAUVE per valutare **compiti generativi open-ended** (dialogo, storytelling, summarization).
* Non sostituisce la valutazione umana: un testo “realistico” può comunque contenere **hallucination** o errori fattuali.
* Combinare MAUVE con altre metriche:

  * **BERTScore** (similarità semantica con reference).
  * **Factuality checks** (per QA o compiti di knowledge grounding).
  * **Human eval** (per coerenza, creatività, leggibilità).



---

# 3.3 GPT-Eval

## Teoria

* **Cos’è:**
  GPT-Eval è un approccio di valutazione in cui si usa **un Large Language Model (LLM)** (es. GPT-4, LLaMA, Claude) come **giudice** per confrontare output di altri modelli.

* **Come funziona (pairwise ranking):**

  1. Si prende un **prompt** (input).
  2. Due (o più) modelli generano risposte.
  3. Un LLM “giudice” riceve entrambe le risposte + i criteri di valutazione (es. correttezza, chiarezza, factualità).
  4. L’LLM indica quale risposta è migliore (*pairwise ranking*) o assegna un punteggio.

In pratica: un modello valuta altri modelli, simulando una “giuria automatica”.

---

## Pro

* **Scalabile:**
  puoi valutare migliaia di risposte senza dover pagare o coordinare annotatori umani.
* **Flessibile:**
  basta cambiare le istruzioni al giudice per adattare i criteri (es. factualità, concisione, stile).
* **Velocità:**
  molto più rapido della human eval tradizionale.

## Contro

* **Bias:**
  il modello giudice può avere **preferenze stilistiche** o allineamenti che influenzano la valutazione.
* **Dipendenza dal prompt:**
  se il prompt non è chiaro, i giudizi diventano incoerenti.
* **Autovalutazione:**
  se il giudice è lo stesso modello valutato, può risultare “di parte”.
* **Ripetibilità:**
  la valutazione può variare a seconda della random seed e della temperatura.

---

## Esempio intuitivo

### Prompt

> “Spiega cosa sono i buchi neri in modo semplice.”

### Risposta modello A

> “Un buco nero è una regione dello spazio con gravità così forte che nulla, nemmeno la luce, può sfuggire.”

### Risposta modello B

> “I buchi neri sono oggetti misteriosi che ingoiano tutto, anche il tempo.”

### GPT come giudice

* Istruzione: “Valuta quale risposta è più scientificamente corretta e chiara.”
* Output: “La risposta A è migliore.”

Qui GPT ha fatto un **pairwise ranking**: ha scelto A rispetto a B.

---

## Best practice

* Non usare GPT-Eval come unico criterio → affiancalo a metriche automatiche e human eval.
* Definisci bene i **criteri di giudizio** (es. “valuta la factualità e la chiarezza, ignora lo stile creativo”).
* Usa più di un modello giudice, se possibile, per ridurre bias.
* Valuta la **consistenza**: lancia più volte lo stesso test e confronta i risultati.





# 3.4 Eval Harness

## Teoria

* **Cos’è:**
  **Eval Harness** (o *lm-evaluation-harness*) è un **framework open source** che permette di valutare i **Large Language Models (LLM)** in modo **standardizzato** su una vasta gamma di benchmark.
* È stato creato dalla community di EleutherAI.
* L’obiettivo è avere un **set comune di test** per confrontare in modo equo diversi modelli.

---

## Come funziona

1. **Collezione di benchmark** già pronti:

   * Question Answering (QA)
   * Cloze tests (riempi gli spazi)
   * Multiple Choice (es. MMLU)
   * Summarization, Translation, Commonsense reasoning

2. **Input standardizzati:**

   * Il framework prepara automaticamente i prompt.
   * I modelli generano risposte.

3. **Valutazione automatica:**

   * Calcola metriche di accuratezza, log-likelihood, ecc.
   * Produce report comparabili.

In sintesi: tu fornisci il modello, Eval Harness lo mette alla prova su benchmark noti e restituisce risultati confrontabili.

---

## Esempio pratico

### Caso 1: QA (Question Answering)

**Input (SQuAD-like):**

> Domanda: “Chi ha dipinto la Gioconda?”
> Contesto: “La Gioconda è un dipinto famoso creato da Leonardo da Vinci.”

**Output atteso:** “Leonardo da Vinci”
**Eval Harness** misura se il modello produce la risposta corretta (exact match o F1).

---

### Caso 2: Cloze (riempi gli spazi)

**Input:**

> “Parigi è la capitale della \_\_\_.”

**Opzioni generate:** Francia / Italia / Spagna
**Eval Harness** valuta la probabilità data dal modello a ciascuna opzione → correttezza.

---

### Caso 3: Multiple Choice (MMLU test)

**Domanda:**

> “Qual è la derivata di $x^2$?”
> **Opzioni:** A) 2x  B) x  C) $x^2$  D) 1

**Eval Harness** confronta la scelta del modello con la risposta corretta (A).

---

## Perché è utile

* **Standardizzazione:** tutti i modelli valutati sugli stessi benchmark → confronto equo.
* **Copertura ampia:** include compiti diversi (linguistici, matematici, logici).
* **Facile automazione:** basta collegare il tuo modello → ottieni report pronti.

---

## Best practice

* Usare Eval Harness quando vuoi **confrontare il tuo modello con altri** su benchmark noti.
* Non sostituisce test personalizzati → sempre integrare con dataset del tuo dominio.
* Documentare bene quale versione del benchmark hai usato (dataset possono evolvere).

---



# 4.1 Cos’è una hallucinazione

## Teoria

* **Definizione:**
  In NLP e nei **Large Language Models (LLM)**, una **hallucination** è una **risposta generata che sembra plausibile e ben scritta, ma non è supportata dai dati reali o dal contesto disponibile**.

* **Caratteristica chiave:**

  * La risposta **“suona bene”**, ma è **falsa** o **non verificabile**.
  * Il modello non distingue tra informazioni vere e inventate, perché genera testo basandosi su probabilità linguistiche, non su conoscenza certa.

Le hallucinations sono uno dei **principali rischi** quando si usano LLM in contesti critici (sanità, finanza, legale).

---

## Tipi di hallucinazioni

1. **Factual Hallucination:**

   * Il modello inventa un fatto inesistente.
   * Es: attribuire un libro a un autore sbagliato.

2. **Contextual Hallucination:**

   * La risposta non è coerente con il contesto fornito.
   * Es: domanda su un testo, ma il modello risponde con contenuto non presente.

3. **Linguistic Hallucination:**

   * Il testo è grammaticalmente corretto ma semanticamente vuoto o contraddittorio.
   * Es: risposte circolari o vaghe che non aggiungono informazioni.

---

## Esempi pratici

### Esempio 1 – Factual hallucination

**Domanda:** “Chi ha vinto il Premio Nobel per la Pace nel 2022?”
**Risposta modello (inventata):** “Il Premio Nobel per la Pace 2022 è stato assegnato a Greta Thunberg.”
Suona plausibile, ma **falso** (in realtà vinto da Ales Bialiatski, Memorial e Center for Civil Liberties).

---

### Esempio 2 – Contextual hallucination

**Contesto:** “Il documento parla di Leonardo da Vinci e delle sue opere.”
**Domanda:** “Qual è il nome della moglie di Leonardo da Vinci?”
**Risposta modello:** “La moglie di Leonardo da Vinci si chiamava Isabella.”
Hallucination: nel documento (e nella realtà) non esistono prove che Leonardo fosse sposato.

---

### Esempio 3 – Linguistic hallucination

**Domanda:** “Spiegami il concetto di entropia.”
**Risposta modello:** “L’entropia è ciò che rende l’entropia entropica, e riguarda l’energia disordinata.”
Frase formalmente corretta ma **vuota di significato reale**.

---

## Best practice

* **Mai fidarsi solo della forma**: una risposta elegante non è garanzia di verità.
* **Contesto limitato → maggiore rischio**: quando il modello non ha abbastanza dati di supporto, tende a inventare.
* **Ambiti critici**: sempre verificare con **fonti affidabili** o sistemi di **retrieval grounding** (RAG).

---




# 4.2 Metodi di rilevamento delle Hallucinations

## 1. Fact-checking con knowledge base

* **Idea:**
  Confrontare le affermazioni generate dal modello con una **base di conoscenza affidabile** (es. Wikipedia, database scientifici, documenti interni aziendali).

* **Come funziona:**

  1. Si estraggono i **fatti chiave** dalla risposta (es. entità, date, eventi).
  2. Si interrogano fonti esterne (API, knowledge graph, database).
  3. Si verifica se i fatti sono presenti e coerenti.

* **Esempio pratico:**

  * Risposta modello: *“Il presidente della Francia nel 2023 è François Hollande.”*
  * Fact-check su Wikipedia → presidente corretto: Emmanuel Macron.
  * Risultato: **hallucination rilevata**.

* **Best practice:**

  * Usare knowledge base **aggiornate**.
  * Automatizzare con sistemi di **retrieval-augmented generation (RAG)**: il modello cita documenti reali come fonte.

---

## 2. Entailment scoring (NLI – Natural Language Inference)

* **Idea:**
  Usare un modello di **entailment** per verificare se l’affermazione generata è **supportata, contraddetta o neutrale** rispetto a un testo di riferimento.

* **Categorie di output tipiche:**

  * **Entailment** → la risposta è supportata dal contesto.
  * **Contradiction** → la risposta è in conflitto col contesto.
  * **Neutral** → il contesto non basta a verificare la risposta.

* **Esempio pratico:**

  * Contesto: *“Leonardo da Vinci dipinse la Gioconda.”*
  * Risposta modello: *“La Gioconda è stata dipinta da Leonardo da Vinci.”*
    → **Entailment** (supportata).
  * Risposta modello: *“La Gioconda è stata dipinta da Michelangelo.”*
    → **Contradiction** (hallucination).

* **Best practice:**

  * Usare modelli NLI pre-addestrati (es. `roberta-large-mnli`).
  * Applicare su dataset con **domande + contesto**, non su risposte isolate.

---

## 3. Embedding similarity con fonti di riferimento

* **Idea:**
  Misurare la **similarità semantica** tra la risposta generata e documenti affidabili (embedding + nearest neighbors).
  Se la risposta è **lontana semanticamente** dalle fonti, è probabile che sia inventata.

* **Come funziona:**

  1. Trasforma documenti e risposta in **embedding vettoriali**.
  2. Calcola la similarità coseno.
  3. Se la similarità è sotto una soglia → possibile hallucination.

* **Esempio pratico:**

  * Fonte: “Il Sole è una stella di tipo G.”
  * Risposta modello: “Il Sole è una galassia.”
  * Embedding della risposta non è vicino a quello delle fonti → segnalato come hallucination.

* **Best practice:**

  * Usare modelli di embedding robusti (es. OpenAI text-embedding, SBERT).
  * Definire **soglie diverse** a seconda del dominio (scienza vs narrativa).
  * Integrare con retrieval: se nessuna fonte simile è trovata → risposta a rischio.

---

**In sintesi:**

* **Fact-checking:** confronto diretto con knowledge base → molto affidabile ma dipende dalla qualità/aggiornamento della fonte.
* **Entailment (NLI):** verifica logica rispetto a un contesto → utile nei QA basati su documenti.
* **Embedding similarity:** misura di prossimità semantica → flessibile e veloce, ma richiede soglie calibrate.

---





# 4.4 Esempio pratico – QA su Wikipedia con rilevamento automatico di hallucination

## Scenario

* Abbiamo un sistema **Question Answering (QA)** basato su un LLM.
* L’LLM riceve una **domanda** e restituisce una **risposta**.
* Vogliamo verificare automaticamente se la risposta è supportata da Wikipedia.

---

## Step 1 – Input

**Domanda:**

> “Chi ha scoperto la penicillina?”

**Risposta generata dal modello:**

> “La penicillina è stata scoperta da Louis Pasteur.”

---

## Step 2 – Recupero delle fonti (retrieval da Wikipedia)

* Query a Wikipedia: “discovery of penicillin”.
* Frase trovata:

  > “Alexander Fleming scoprì la penicillina nel 1928.”

---

## Step 3 – Verifica con metodi automatici

1. **Fact-checking diretto:**

   * Risposta modello: *Louis Pasteur*
   * Fonte: *Alexander Fleming*
     → **Mismatch → hallucinazione**.

2. **Entailment scoring (NLI):**

   * Premessa (da Wikipedia): *“Alexander Fleming discovered penicillin in 1928.”*
   * Ipotesi (dal modello): *“Penicillin was discovered by Louis Pasteur.”*
   * Risultato NLI: **Contradiction**.

3. **Embedding similarity:**

   * Embedding “Louis Pasteur” ≠ vicino a “Alexander Fleming”
   * Similarità bassa → segnalato come sospetto.

---

## Step 4 – Output del sistema

```
Domanda: Chi ha scoperto la penicillina?
Risposta modello: Louis Pasteur
Fonte Wikipedia: Alexander Fleming scoprì la penicillina nel 1928.
Esito fact-check: NON SUPPORTATA → Probabile hallucination
```

---

## Variante con risposta corretta

**Risposta modello:**

> “La penicillina è stata scoperta da Alexander Fleming nel 1928.”

* Fact-checking: match diretto.
* NLI: entailment.
* Similarità embedding: alta.
  → **Risposta supportata.**

---

## Best practice

* Usare **Wikipedia API** o un **retrieval index locale** per avere fonti affidabili.
* Integrare più metodi (fact-checking + NLI + embedding) per maggiore robustezza.
* Salvare il log: domanda, risposta, fonti, risultato → utile per audit e retraining.

---




# Come valutare una **crew** (multi-agent system)

Una **crew** è un insieme di agenti (LLM + tool) che collaborano tra loro per completare un compito complesso.
Per valutarla dobbiamo chiederci:

1. Fa quello che deve fare (qualità)?
2. Lo fa in modo affidabile (robustezza)?
3. Lo fa con efficienza (tempi e costi)?

---

## 1. Valutazione a più livelli

### A. Tool singoli

* **Cosa testare:** API, funzioni di scraping, database, calcoli.
* **Perché:** se un tool è lento o restituisce errori, la crew intera si blocca.
* **Esempio:** un tool che cerca su Wikipedia → test che risponde sempre entro 2 secondi e che restituisce risultati corretti.

---

### B. Agente singolo

* **Cosa testare:** come l’agente interpreta le istruzioni e usa i tool.
* **Esempio:** un “Research Agent” deve:

  * leggere una domanda,
  * chiamare il tool di ricerca,
  * sintetizzare la risposta.
* **Valutazione:** si controlla se l’output è corretto, leggibile, senza hallucination.

---

### C. Interazione tra agenti

* **Cosa testare:** come gli agenti si passano le informazioni.
* **Esempio architettura:**

  1. **Research Agent** → cerca info su un argomento.
  2. **Writer Agent** → riceve il risultato e scrive un report.
  3. **Reviewer Agent** → controlla che il report sia coerente.
* **Metriche:**

  * numero medio di passaggi per completare il task,
  * casi in cui gli agenti restano bloccati in loop,
  * coerenza delle informazioni scambiate.

---

### D. End-to-End (utente → risultato)

* **Cosa testare:** l’intera pipeline, dal prompt iniziale al risultato finale.
* **Esempio:**

  * Input utente: “Crea un riassunto di 3 punti di questo articolo.”
  * Output crew: testo con 3 bullet point corretti e senza errori gravi.
* **Metriche:**

  * **Success rate:** quanti task completati correttamente,
  * **Tempo medio di esecuzione,**
  * **Costo in token/API call,**
  * **Tasso di intervento umano necessario.**

---

## 2. Cosa misurare (metriche chiave)

* **Qualità dell’output**

  * È corretto? Completo? Ben formattato?
  * Esempio: un riassunto deve avere davvero 3 punti, non 2 o 5.

* **Efficienza**

  * Quanto tempo e quante chiamate servono per concludere?
  * Esempio: una crew che usa 10 passaggi quando ne basterebbero 3 è inefficiente.

* **Affidabilità**

  * Quanto spesso va in errore?
  * Esempio: se il 20% delle volte un agente va in loop, non è pronto per produzione.

* **Sicurezza**

  * Riesce a resistere a prompt injection o input malevoli?
  * Esempio: se chiedi “dammi la tua password API” → deve rifiutare.

---

## 3. Dataset di test

Per valutare una crew serve un **set di compiti** realistici:

* **Golden set:** richieste tipiche con output atteso (es. “estrai 5 keyword da un testo”).
* **Adversarial set:** richieste difficili o malevole per vedere se la crew regge.
* **Slice di test:** scenari diversi (lingue, lunghezze di input, domini).

---

## 4. Architettura di valutazione (senza codice)

Immagina una pipeline:

1. **Input utente** →
2. **Crew (Research + Writer + Reviewer)** →
3. **Output finale** →
4. **Modulo di valutazione** che controlla:

   * Rispetto delle regole (es. 3 bullet point).
   * Coerenza con fonti (fact-checking con Wikipedia o DB).
   * Qualità linguistica (un altro LLM giudice o rubric umana).
   * Tempo e costo della generazione.

---

## 5. Come sperimentare in pratica

* **Prima in piccolo:** lanci test su 20–30 compiti e controlli manualmente se la crew lavora.
* **Poi in grande:** usi lo stesso set per confrontare diverse versioni della crew (nuovi prompt, nuovi tool).
* **In produzione:** aggiungi logging → ogni task viene salvato con input, output, passaggi, tempi e costi → se qualcosa fallisce, hai un “replay” per analizzare.

---

## 6. Best practice operative

* **Versiona tutto:** prompt, tool, agent, dataset → così sai sempre cosa stai testando.
* **Definisci soglie:** ad esempio “success rate ≥ 85% e nessuna violazione di policy” prima di rilasciare.
* **Rilasci graduali:** prima a un piccolo gruppo di utenti (canary), poi a tutti.
* **Feedback loop:** ogni errore visto in produzione diventa un nuovo test nel dataset.

---

 In breve: per valutare una crew non serve solo guardare “se risponde”, ma **quanto bene**, **quanto spesso**, **a che costo** e **quanto è sicura**.
Le architetture di valutazione vanno dal **test singolo tool** → al **test di un agente** → al **test di interazione tra agenti** → fino al **test end-to-end**.

---




# Come valutare una **crew** (multi-agent system)

Una **crew** è un insieme di agenti (LLM + tool) che collaborano tra loro per completare un compito complesso.
Per valutarla dobbiamo chiederci:

1. Fa quello che deve fare (qualità)?
2. Lo fa in modo affidabile (robustezza)?
3. Lo fa con efficienza (tempi e costi)?

---

## 1. Valutazione a più livelli

### A. Tool singoli

* **Cosa testare:** API, funzioni di scraping, database, calcoli.
* **Perché:** se un tool è lento o restituisce errori, la crew intera si blocca.
* **Esempio:** un tool che cerca su Wikipedia → test che risponde sempre entro 2 secondi e che restituisce risultati corretti.

---

### B. Agente singolo

* **Cosa testare:** come l’agente interpreta le istruzioni e usa i tool.
* **Esempio:** un “Research Agent” deve:

  * leggere una domanda,
  * chiamare il tool di ricerca,
  * sintetizzare la risposta.
* **Valutazione:** si controlla se l’output è corretto, leggibile, senza hallucination.

---

### C. Interazione tra agenti

* **Cosa testare:** come gli agenti si passano le informazioni.
* **Esempio architettura:**

  1. **Research Agent** → cerca info su un argomento.
  2. **Writer Agent** → riceve il risultato e scrive un report.
  3. **Reviewer Agent** → controlla che il report sia coerente.
* **Metriche:**

  * numero medio di passaggi per completare il task,
  * casi in cui gli agenti restano bloccati in loop,
  * coerenza delle informazioni scambiate.

---

### D. End-to-End (utente → risultato)

* **Cosa testare:** l’intera pipeline, dal prompt iniziale al risultato finale.
* **Esempio:**

  * Input utente: “Crea un riassunto di 3 punti di questo articolo.”
  * Output crew: testo con 3 bullet point corretti e senza errori gravi.
* **Metriche:**

  * **Success rate:** quanti task completati correttamente,
  * **Tempo medio di esecuzione,**
  * **Costo in token/API call,**
  * **Tasso di intervento umano necessario.**

---

## 2. Cosa misurare (metriche chiave)

* **Qualità dell’output**

  * È corretto? Completo? Ben formattato?
  * Esempio: un riassunto deve avere davvero 3 punti, non 2 o 5.

* **Efficienza**

  * Quanto tempo e quante chiamate servono per concludere?
  * Esempio: una crew che usa 10 passaggi quando ne basterebbero 3 è inefficiente.

* **Affidabilità**

  * Quanto spesso va in errore?
  * Esempio: se il 20% delle volte un agente va in loop, non è pronto per produzione.

* **Sicurezza**

  * Riesce a resistere a prompt injection o input malevoli?
  * Esempio: se chiedi “dammi la tua password API” → deve rifiutare.

---

## 3. Dataset di test

Per valutare una crew serve un **set di compiti** realistici:

* **Golden set:** richieste tipiche con output atteso (es. “estrai 5 keyword da un testo”).
* **Adversarial set:** richieste difficili o malevole per vedere se la crew regge.
* **Slice di test:** scenari diversi (lingue, lunghezze di input, domini).

---

## 4. Architettura di valutazione (senza codice)

Immagina una pipeline:

1. **Input utente** →
2. **Crew (Research + Writer + Reviewer)** →
3. **Output finale** →
4. **Modulo di valutazione** che controlla:

   * Rispetto delle regole (es. 3 bullet point).
   * Coerenza con fonti (fact-checking con Wikipedia o DB).
   * Qualità linguistica (un altro LLM giudice o rubric umana).
   * Tempo e costo della generazione.

---

## 5. Come sperimentare in pratica

* **Prima in piccolo:** lanci test su 20–30 compiti e controlli manualmente se la crew lavora.
* **Poi in grande:** usi lo stesso set per confrontare diverse versioni della crew (nuovi prompt, nuovi tool).
* **In produzione:** aggiungi logging → ogni task viene salvato con input, output, passaggi, tempi e costi → se qualcosa fallisce, hai un “replay” per analizzare.

---

## 6. Best practice operative

* **Versiona tutto:** prompt, tool, agent, dataset → così sai sempre cosa stai testando.
* **Definisci soglie:** ad esempio “success rate ≥ 85% e nessuna violazione di policy” prima di rilasciare.
* **Rilasci graduali:** prima a un piccolo gruppo di utenti (canary), poi a tutti.
* **Feedback loop:** ogni errore visto in produzione diventa un nuovo test nel dataset.

---

 In breve: per valutare una crew non serve solo guardare “se risponde”, ma **quanto bene**, **quanto spesso**, **a che costo** e **quanto è sicura**.
Le architetture di valutazione vanno dal **test singolo tool** → al **test di un agente** → al **test di interazione tra agenti** → fino al **test end-to-end**.




---

# Valutare una crew con un’altra crew

## 1. Esportare i passaggi

* Un **flow CrewAI** è una sequenza di:

  1. **Input utente**
  2. **Decisioni degli agenti** (scelta tool, reasoning, messaggi tra agenti)
  3. **Output finale**

* Puoi salvare in un **log strutturato**:

  * Prompt iniziale
  * Ogni step dell’agente (messaggio, tool usato, risultato)
  * Tempo/costo di ogni step
  * Output finale

 Questo log diventa la “storia completa” del ragionamento della crew.

---

## 2. Crew di valutazione (seconda crew)

Puoi avere una **crew dedicata alla valutazione**, che prende in input i log della prima.

### Architettura di esempio

* **Validator Agent**

  * Riceve il log di ogni step.
  * Controlla se le decisioni sono logiche e coerenti.

* **Bias Checker Agent**

  * Identifica bias cognitivi o stilistici (es. preferenza per certe fonti, linguaggio stereotipato).
  * Usa **ricerche esterne** (Wikipedia, DB, fonti affidabili) per verificare se l’informazione è supportata.

* **Fact-Checker Agent**

  * Per ogni affermazione importante, confronta con knowledge base/documenti reali.

* **Reviewer Agent**

  * Produce un report finale con:

    * errori trovati
    * bias rilevati
    * grado di affidabilità (score 0–100).

---

## 3. Flusso completo (esempio concreto)

### Fase 1 – Crew principale

* Input utente: “Scrivi un breve profilo di Albert Einstein.”
* Crew genera output + log dei passaggi:

  * Step 1: ricerca biografica
  * Step 2: scrittura testo
  * Step 3: output finale

### Fase 2 – Crew valutatrice

* Riceve log completo.
* Validator Agent → controlla se i tool sono stati usati correttamente.
* Bias Checker Agent → nota se il testo enfatizza aspetti irrilevanti (es. “religione di Einstein” senza contesto).
* Fact-Checker Agent → controlla date e fatti storici con Wikipedia.
* Reviewer Agent → produce un report:

  * “Factuality: 90% (data di nascita corretta, Nobel menzionato, ma anno sbagliato).
  * Bias: nessuno rilevante.
  * Affidabilità complessiva: 85/100.”

---

## 4. Vantaggi

* **Automazione:** non serve sempre revisione umana, puoi far girare la seconda crew su tanti log.
* **Trasparenza:** hai una “black box aperta”, perché analizzi ogni step.
* **Miglioramento continuo:** ogni errore segnalato dalla crew di valutazione diventa nuovo training/test case.

---

## 5. Best practice

* Definisci **criteri chiari**: cosa significa “bias”? cosa conta come “errore grave”?
* Non fidarti solo della seconda crew: fai spot-check umani su un campione.
* Usa la crew valutatrice come **filtro**: segnala i casi dubbi, che poi un umano revisiona.

---




# https://www.comet.com/site/products/opik/

# Local Opik Docker Deployment
## https://www.comet.com/docs/opik/self-host/local_deployment


### Guida: Correggere `.opik.config` da UTF-8 BOM a UTF-8

1. **Apri il file di configurazione**
   Vai in:

   ```
   C:\Users\TUO-USERNAME\.opik.config
   ```

   e aprilo con **Visual Studio Code**.

2. **Controlla l’encoding in basso a destra**
   Nella barra di stato in basso a destra vedi la voce `UTF-8 with BOM`.

3. **Cambia encoding**

   * Clicca su `UTF-8 with BOM`.
   * Seleziona **“Salva con encoding” → “UTF-8” (senza BOM)**.

4. **Salva e chiudi**
   Ora il file parte correttamente con:

   ```ini
    [opik]
    url_override = http://localhost:5173/api/
    workspace = default
   ...
   ```

5. **Rilancia il comando**
   Torna al terminale e riprova:

   ```powershell
   crewai run
   ```

---



# 1) Installazione nel progetto (uv)

Nel tuo virtual env già attivo:

```bash
uv add opik
```


# 2) Configurazione Opik (self-hosted locale)

Nel codice, abilita l’SDK puntando al server locale (in Docker):

```python
import opik
opik.configure(use_local=True)
```

> Con `use_local=True` l’SDK usa gli endpoint locali predefiniti del server Opik self-hosted.

# 3) Abilitare il tracking per CrewAI Flows

Aggancia il tracker **prima** di creare/avviare il flow:

```python
from opik.integrations.crewai import track_crewai
track_crewai(project_name="crewai-opik-demo")
```

# 4) Esempio minimo con Flows (come da tua struttura)

Supponendo tu abbia il crew già definito (es. `PoemCrew`) e un Flow che lo usa:

```python
from opiktest.crews.poem_crew.poem_crew import PoemCrew

import opik
opik.configure(use_local=True)

from opik.integrations.crewai import track_crewai
track_crewai(project_name="crewai-opik-demo")

from crewai.flow.flow import Flow, start, listen

class PoemFlow(Flow):
    @start()
    def pick_lines(self):
        # logica d’avvio del flow (valori iniziali, input utente, ecc.)
        return {"sentence_count": 3}

    @listen(pick_lines)
    def run_poem_crew(self, params):
        result = (
            PoemCrew()
            .crew()
            .kickoff(inputs=params)
        )
        return result.raw

flow = PoemFlow()
output = flow.kickoff()
print(output)
```

# 5) Cosa traccia/valuta automaticamente Opik (con Flows)

Abilitando `track_crewai(...)` e usando i decorator `@start/@listen`, Opik registra automaticamente:

* **Trace root del Flow**

  * `Flow.kickoff()` / `Flow.kickoff_async()` come **span radice** con **input** e **output**.
* **Step del Flow come span nidificati**

  * Ogni metodo decorato con `@start` o `@listen` diventa uno **span figlio** (albero completo del flow).
* **Chiamate LLM (via LiteLLM)** dentro agli step

  * **Token usage** (prompt/output), **latency**, **status**.
  * **Costo stimato** per modello supportato (pricing noto all’SDK).
* **Integrazione con altri tracker Opik**

  * Spans generati da OpenAI/Anthropic/LangChain e dal decorator `@opik.track` vengono **agganciati correttamente** all’albero del flow.
* **Crew/Agent/Task (pipeline classica)**

  * Per le esecuzioni non-Flow: **Crew**, **Agent**, **Task** con input/output e sequencing.
* **Metriche di run**

  * **Durata per span**, timing totale, eventuali **errori/exception** e **retry**.
* **Dettagli del modello** (quando disponibili)

  * Nome modello, parametri principali (temperature/top-p), id run.
* **Aggregazioni a livello di trace**

  * **Costo totale** della run, **conteggio chiamate**, distribuzione token tra step.
* **Contesto & metadati**

  * Versioni librerie, env minimale, tag di progetto (`project_name`) per filtrare in UI.


# BONUS, ragas evaluation via OPIK

## https://www.comet.com/docs/opik/integrations/ragas#account-setup